In [7]:
import os
os.makedirs("/home/jovyan/state/spark-events", exist_ok=True)

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("proj1")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "8")
    .config("spark.eventLog.enabled", "true")
    .config("spark.eventLog.dir", "file:/home/jovyan/state/spark-events")
    .getOrCreate()
)


In [8]:


spark_ok = True
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import avg
except Exception as e:
    spark_ok = False
    print("PySpark not available:", e)

if spark_ok:
    spark = (
        SparkSession.builder
        .appName("proj1")
        .master("local[*]")
        .config("spark.sql.shuffle.partitions", "8")
        .getOrCreate()
    )
else:
    print('spark not ok')

In [9]:
spark.range(0, 1_000_000).count()


1000000

In [11]:
from pyspark.sql import functions as F

inbox = "/home/jovyan/work/inbox"
lookup_path = "/home/jovyan/work/inbox/taxi_zone_lookup.parquet"

trips = spark.read.parquet(f"{inbox}/yellow_tripdata_2025-01.parquet")
zones = spark.read.parquet(lookup_path)

# Inspect
trips.printSchema()
zones.show(5, truncate=False)


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)

+----------+-------------+-----------------------+------------+
|LocationID